# Magentic-One Research Pipeline with Bing Search

🔍 **Advanced Multi-Agent Research System**

This tutorial demonstrates:
- **Magentic-One orchestration** with complex research tasks
- **Bing Search-powered Azure AI Agents** for web research
- **Multi-step information retrieval** with intelligent coordination
- **Complex query decomposition** and fact verification

**Architecture:**
```
┌─────────────────────────────────────────────────────────────┐
│                    Magentic Orchestration                   │
│                (StandardMagenticManager)                    │
└─────────────────────┬───────────────────────────────────────┘
                      │
    ┌─────────────────┼─────────────────┐
    │                 │                 │
┌───▼────┐    ┌──────▼──────┐    ┌─────▼─────┐
│Research│    │   Fact      │    │ Report    │
│ Agent  │    │ Checker     │    │Generator  │
│(Bing)  │    │  (Bing)     │    │  (Azure)  │
└───┬────┘    └──────┬──────┘    └─────┬─────┘
    │                │                 │
┌───▼────┐    ┌──────▼──────┐    ┌─────▼─────┐
│ Web    │    │Cross-verify │    │  Final    │
│Search  │    │Information  │    │ Report    │
│Tool    │    │   Tool      │    │Generator  │
└────────┘    └─────────────┘    └───────────┘
```

**Use Case**: *"From which university did the chief of staff of the 43rd president of the US get his degree from? And in what specialty?"*

This requires:
1. Identifying the 43rd president of the US
2. Finding who was their chief of staff
3. Researching the chief of staff's educational background
4. Cross-verifying the information
5. Generating a comprehensive report

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name
- `AZURE_FOUNDRY_PROJECT_ENDPOINT`: Your Azure AI Foundry project endpoint

**Azure Configuration:**
- Uses **Azure AI Agents** with Bing Search grounding
- Uses **AzureChatCompletion** service for orchestration
- Automatically discovers Bing Search connections
- Compatible with Azure AI Studio deployments

**Research Architecture:**
1. **Research Agent**: Primary web search and information gathering
2. **Fact Checker Agent**: Cross-verification of found information
3. **Report Generator Agent**: Synthesis and final report creation
4. **Magentic Orchestrator**: Coordinates complex multi-step research

**Complex Research Flow:**
- Decomposes complex questions into searchable parts
- Performs sequential web searches with context awareness
- Cross-verifies information from multiple sources
- Generates comprehensive, cited reports


### Configuration Steps of the Grounding with Bing Search Connection

Please follow the below two links to properly configure the Grounding with Bing Search Connection in Azure AI Studio:

- [Create a Grounding with Bing Search Connection](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/bing-grounding)
- [Create a Grounding with Bing Search Connection in Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/bing-code-samples?pivots=portal)

In [1]:
# Install required packages for Magentic orchestration with Bing search
# !pip install azure-ai-agents azure-identity semantic-kernel azure-ai-projects

import os
import json
import asyncio
from datetime import datetime, timedelta
from typing import List, Optional

# Azure AI Agents and Semantic Kernel imports
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from semantic_kernel.agents import (
    Agent,
    AzureAIAgent,
    MagenticOrchestration,
    StandardMagenticManager,
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Azure AI Agents client imports
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import MessageRole, BingGroundingTool

print("✅ Packages imported successfully!")
print(f"📅 Tutorial started at: {datetime.now()}")
print("🔍 Using Azure AI Agents with Bing Search for Research Pipeline")

✅ Packages imported successfully!
📅 Tutorial started at: 2025-06-10 15:38:10.810122
🔍 Using Azure AI Agents with Bing Search for Research Pipeline


## 🌐 Step 1: Discover Bing Search Connection

Let's discover and configure the Bing Search connection for our research agents.

In [2]:
# Discover Bing Search connection
async def discover_bing_connection() -> Optional[str]:
    """Discover and return the Bing Search connection ID"""
    
    print("🔍 Discovering Bing Search connections...")
    
    try:
        # Get project endpoint from environment
        project_endpoint = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT") or os.environ.get("PROJECT_ENDPOINT")
        
        if not project_endpoint:
            print("❌ No project endpoint found in environment variables")
            return None
            
        async with DefaultAzureCredential() as creds:
            project_client = AIProjectClient(
                endpoint=project_endpoint,
                credential=creds,
                api_version="2025-05-15-preview"
            )
            
            # List all connections and find Bing Search
            bing_connections = []
            async for connection in project_client.connections.list():
                print(f"📋 Found connection: {connection.name} (Type: {connection.type})")
                if connection.type == "GroundingWithBingSearch":
                    bing_connections.append(connection)
                    print(f"✅ Bing Search connection found: {connection.id}")
            
            if bing_connections:
                selected_connection = bing_connections[0]  # Use first available
                print(f"🎯 Selected Bing connection: {selected_connection.name}")
                return selected_connection.id
            else:
                print("❌ No Bing Search connections found")
                print("💡 Please create a Bing Search connection in Azure AI Studio")
                return None
                
    except Exception as e:
        print(f"❌ Error discovering Bing connection: {e}")
        return None

# Test connection discovery
print("🌐 Testing Bing Search connection discovery...")

🌐 Testing Bing Search connection discovery...


In [ ]:
# Discover the Bing connection
bing_connection_id = await discover_bing_connection()

if bing_connection_id:
    print(f"✅ Bing connection ready: {bing_connection_id[-50:]}...")  # Show last 50 chars for security
else:
    print("⚠️ No Bing connection available - will use basic agents without web search")

## 🤖 Step 2: Create Research-Specialized Agents

We'll create three specialized agents for complex research tasks:
1. **Research Agent**: Primary web search and information gathering
2. **Fact Checker Agent**: Cross-verification and accuracy checking
3. **Report Generator Agent**: Synthesis and comprehensive reporting

In [4]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print agent messages with enhanced formatting"""
    if message.role != AuthorRole.TOOL:
        print(f"\n🎭 **{message.name}**")
        # Truncate very long responses for readability
        content = message.content
        if len(content) > 500:
            content = content[:500] + "... [content truncated]"
        print(f"   {content}")
        print("-" * 60)

async def create_research_agents(bing_connection_id: Optional[str] = None) -> List[Agent]:
    """Create three specialized agents for complex research tasks"""
    
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]
    agents = []
    
    print("🔍 Creating Research Pipeline Agents...")
    
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # Prepare Bing tool if connection is available
        bing_tools = []
        if bing_connection_id:
            bing_tool = BingGroundingTool(connection_id=bing_connection_id)
            bing_tools = bing_tool.definitions
            print("✅ Bing Search tools configured")
        else:
            print("⚠️ No Bing connection - agents will work without web search")
        
        # 1. Research Agent - Primary information gatherer
        print("🔎 Creating Research Agent...")
        research_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="research_agent",
            description="Expert research agent with web search capabilities",
            instructions="""
            You are an expert research agent specialized in finding accurate information online.
            
            Your capabilities:
            - Break down complex questions into searchable components
            - Perform systematic web searches using Bing
            - Extract relevant facts and details from search results
            - Maintain context between related searches
            - Focus on authoritative sources (Wikipedia, official sites, news)
            
            When given a complex research task:
            1. Analyze the question and identify key information needed
            2. Break it into sequential search steps
            3. Perform searches systematically, building on previous findings
            4. Extract specific facts, names, dates, and details
            5. Always cite your sources with URLs when possible
            
            Example approach for "Who was the chief of staff of the 43rd president?":
            1. Search: "43rd president of United States"
            2. Search: "[President name] chief of staff"
            3. Search: "[Chief of staff name] education university degree"
            
            Be thorough, accurate, and always provide source citations.
            """,
            tools=bing_tools
        )
        
        research_agent = AzureAIAgent(
            client=client,
            definition=research_agent_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        agents.append(research_agent)
        print(f"✅ Research Agent: {research_agent_definition.id}")
        
        # 2. Fact Checker Agent - Verification specialist
        print("✅ Creating Fact Checker Agent...")
        fact_checker_definition = await client.agents.create_agent(
            model=model_deployment,
            name="fact_checker",
            description="Fact verification and cross-checking specialist",
            instructions="""
            You are a fact-checking specialist focused on verifying information accuracy.
            
            Your role:
            - Cross-verify facts from multiple independent sources
            - Check for consistency in dates, names, and details
            - Identify any conflicting information
            - Assess source reliability and credibility
            - Flag uncertain or disputed information
            
            When fact-checking:
            1. Take the research findings from other agents
            2. Perform independent searches to verify key facts
            3. Look for multiple sources confirming the same information
            4. Note any discrepancies or conflicting reports
            5. Assess confidence level in each fact
            6. Provide verification status with source citations
            
            Always be thorough and objective in your verification process.
            """,
            tools=bing_tools
        )
        
        fact_checker = AzureAIAgent(
            client=client,
            definition=fact_checker_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        agents.append(fact_checker)
        print(f"✅ Fact Checker: {fact_checker_definition.id}")
        
        # 3. Report Generator Agent - Synthesis specialist
        print("📊 Creating Report Generator Agent...")
        report_generator_definition = await client.agents.create_agent(
            model=model_deployment,
            name="report_generator",
            description="Comprehensive report generation specialist",
            instructions="""
            You are a report generation specialist who creates comprehensive, well-structured reports.
            
            Your capabilities:
            - Synthesize information from multiple sources and agents
            - Create clear, organized, and comprehensive reports
            - Include proper citations and source references
            - Highlight key findings and important details
            - Note confidence levels and any uncertainties
            
            Report structure should include:
            1. **Executive Summary**: Brief answer to the original question
            2. **Detailed Findings**: Step-by-step research results
            3. **Key Facts**: Important names, dates, and details
            4. **Source Citations**: All sources used with URLs
            5. **Confidence Assessment**: Reliability of the information
            6. **Additional Notes**: Any relevant context or caveats
            
            Always create professional, accurate, and well-cited reports.
            """
            # No Bing tools needed - focuses on synthesis
        )
        
        report_generator = AzureAIAgent(
            client=client,
            definition=report_generator_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        agents.append(report_generator)
        print(f"✅ Report Generator: {report_generator_definition.id}")
    
    print(f"✅ Created {len(agents)} research agents total")
    return agents

print("🔍 Research agents configuration ready")

🔍 Research agents configuration ready


## 🎭 Step 3: Create Magentic Research Orchestration

Now we'll create the Magentic orchestration that will coordinate our research agents to handle complex, multi-step information retrieval tasks.

In [5]:
async def create_research_orchestration(bing_connection_id: Optional[str] = None) -> MagenticOrchestration:
    """Create and configure the research Magentic orchestration"""
    
    print("🎭 Setting up Research Magentic-One orchestration...")
    
    # Create research agents
    agents = await create_research_agents(bing_connection_id)
    
    # Configure Azure Chat Completion for the manager
    manager_azure_chat_completion = AzureChatCompletion()
    
    # Create the Magentic orchestration
    # The StandardMagenticManager coordinates complex research workflows
    research_orchestration = MagenticOrchestration(
        members=agents,
        manager=StandardMagenticManager(
            chat_completion_service=manager_azure_chat_completion
        ),
        agent_response_callback=agent_response_callback,
    )
    
    print(f"✅ Research orchestration created with {len(agents)} agents")
    print("   Agents: " + ", ".join([agent.name for agent in agents]))
    
    return research_orchestration

print("🎯 Research orchestration configuration ready")

🎯 Research orchestration configuration ready


## 🚀 Step 4: Execute Complex Research Task

Now let's execute our complex research use case that requires multiple steps and web searches to answer completely.

**Research Question**: *"From which university did the chief of staff of the 43rd president of the US get his degree from? And in what specialty?"*

This involves:
1. Identifying the 43rd president of the United States
2. Finding who served as their chief of staff
3. Researching the chief of staff's educational background
4. Cross-verifying the information from multiple sources
5. Generating a comprehensive, cited report

In [6]:
async def execute_complex_research_task(bing_connection_id: Optional[str] = None):
    """Execute the complex research task using Magentic orchestration"""
    
    print("🔍 Starting Complex Research Task")
    print("=" * 70)
    
    # Create the orchestration
    orchestration = await create_research_orchestration(bing_connection_id)
    
    # Create and start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Define the complex research task
        research_task = """
        I need you to conduct comprehensive research to answer this complex question:
        
        "From which university did the chief of staff of the 43rd president of the US get his degree from? And in what specialty?"
        
        This requires multiple research steps:
        
        1. Research Agent: 
           - First, identify who was the 43rd president of the United States
           - Then find who served as their chief of staff (there may have been multiple)
           - Research the educational background of the chief of staff(s)
           - Find specific details about their university degree(s) and field of study
        
        2. Fact Checker:
           - Verify the identity of the 43rd president
           - Cross-check the chief of staff information
           - Verify educational credentials from multiple sources
           - Check for accuracy of university names and degree specialties
        
        3. Report Generator:
           - Create a comprehensive report with all findings
           - Include clear citations and source URLs
           - Provide confidence assessments for each fact
           - Structure the information clearly and professionally
        
        Please work together systematically to research this question thoroughly.
        Use web search capabilities to find accurate, up-to-date information.
        """
        
        print("📋 Complex Research Task Definition:")
        print(research_task)
        print("\n" + "=" * 70)
        print("🔍 Beginning Research Orchestration...")
        print("=" * 70)
        
        # Execute the orchestration
        orchestration_result = await orchestration.invoke(
            task=research_task,
            runtime=runtime
        )
        
        # Wait for the results
        result = await orchestration_result.get()

        # Display final results
        print("\n" + "=" * 70)
        print("🎯 RESEARCH ORCHESTRATION COMPLETE")
        print("\n📊 Final Research Report:")
        print(f"\n{result}")
        print("=" * 70)
        
    except Exception as e:
        print(f"❌ Error during orchestration: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Cleanup
        await runtime.stop()
        print("\n🧹 Runtime stopped and resources cleaned up")

print("🌟 Ready to execute complex research task!")
print("This will involve three agents working together through Magentic orchestration.")
print("The task requires multiple web searches and cross-verification of information.")

🌟 Ready to execute complex research task!
This will involve three agents working together through Magentic orchestration.
The task requires multiple web searches and cross-verification of information.


In [7]:
# Execute the complex research task
await execute_complex_research_task(bing_connection_id)

🔍 Starting Complex Research Task
🎭 Setting up Research Magentic-One orchestration...
🔍 Creating Research Pipeline Agents...
✅ Bing Search tools configured
🔎 Creating Research Agent...
✅ Research Agent: asst_XklEv4pFc7LFR8Wq5WPXkXi7
✅ Creating Fact Checker Agent...
✅ Fact Checker: asst_XdHO1GGg1ZNsZo3KsyZsJ7hF
📊 Creating Report Generator Agent...
✅ Report Generator: asst_VY6qGNy18TK7WK9oNGwsqoQm
✅ Created 3 research agents total
✅ Research orchestration created with 3 agents
   Agents: research_agent, fact_checker, report_generator
📋 Complex Research Task Definition:

        I need you to conduct comprehensive research to answer this complex question:

        "From which university did the chief of staff of the 43rd president of the US get his degree from? And in what specialty?"

        This requires multiple research steps:

        1. Research Agent: 
           - First, identify who was the 43rd president of the United States
           - Then find who served as their chief of st